### Introduction

In this laboratory we will use tools and test functions provided by a library pymoo. It is a well recognized optimization library created specifically for multi-objective problems. However, it does contain a wide range of single-objective metaheuristics more than sufficient for this laboratory.

For documentation refer to https://pymoo.org/

### CMA-ES introduction

A robust guide to CMA-ES is available at https://doi.org/10.48550/arXiv.1604.00772

Pymoo does not provide its own implementation of CMA-ES instead it calls the pycma library which is mantained by the author of the algorithm Nikolaus Hansen https://pypi.org/project/cma/. Other notable implementation of this algorithm is available at https://pypi.org/project/cmaes/ library. You can look up the visualizations provided in its documentation to get a better grasp on how different variants of the algorithm work.

In [2]:
import matplotlib.pyplot as plt
import numpy as np

from pymoo.algorithms.soo.nonconvex.cmaes import CMAES
from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.core.callback import Callback
from pymoo.operators.sampling.lhs import LHS
from pymoo.optimize import minimize
from pymoo.problems import get_problem
from pymoo.termination import get_termination
from pymoo.visualization.fitness_landscape import FitnessLandscape

In [4]:
np.linalg.norm([-1, -4, -6, -1, -30, -16])

34.785054261852174

In [ ]:
sphere = get_problem("sphere")

cma = CMAES(x0=np.random.random(sphere.n_var))

res = minimize(sphere,
               cma)

print(f"Best solution found: \nX = {res.X}\nF = {res.F}\nCV= {res.CV}")

### Excercise 1
Test CMA-ES on a 20-dimensional Rastrigin function. Use Callback interface to get best function values for each consecutive generation and plot them on a line plot.

### Excercise 2
Find out how to run BIPOP-CMA-ES using a pymoo library. Then compare it to the standard CMA-ES using the same criteria as in previous task.

### Differential Evolution

A decent open access article is available on wikipedia: https://en.wikipedia.org/wiki/Differential_evolution while a robust review and explanation of different variants up to 2010 is available at https://www.researchgate.net/publication/220380793_Differential_Evolution_A_Survey_of_the_State-of-the-Art

Implementations are among others available at SciPy https://docs.scipy.org/doc/scipy/reference/optimize.html and pymoo https://pymoo.org/. Various more advanced variants can be found at small libraries created by researchers.

#### Possible variants construction and parameterization:
##### Controlling mutation:
best/rand/current-to-best/rand-to-best -> which vectors are compared to create a donor vector

1/2 -> how many vector differences are used to create a donor vector

##### Controlling crossover:
bin/exp -> binomial crossover samples between a value from candidate or original individual separately for every dimension. Exponential crossover samples a cutting point after which a number of values from candidate solutions are chosen

##### Parameters:
population size, F, CR -> F is responsible for scaling a difference between vectors during mutation. Bigger F makes the algorithm more volatile. CR is a probability factor which has impact on crossover. Regardles of variant, smaller CR means that more of candidate vector is used while higher CR responds to more of an original vector remaining.

In [ ]:
problem = get_problem("ackley", n_var=10)


algorithm = DE(
    variant="DE/rand/1/bin",
    sampling=LHS(),
    pop_size=100,
    F=0.5,
    CR=0.2
)

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

### Excercise 3
Choose a test problem and termination criteria. Test Differential Evolution with a few combinations of parameters or variants by reruning the algorithm 10 times for each setup. Compare mean best results using boxplots.

### Excercise 4
Run the algorithm with a population of 20 individuals on a 2-dimensional Sphere function and visualize a few consecutive generations. Use FitnessLandscape feature of pymoo (_type="contour") to visualize the space in which the individuals exist.

############### HINT ###############

<details>
  <summary>Plotting on top of fitness landscape in this specific library is not trivial. If you encounter problems try steps below:</summary>
  
    1. create fitness class
    2. call init_figure method
    3. now you can access ax property and operate on it
    4. call do method to show the plot
</details>

### Excercise 5 - benchmark
Compare GA, PSO, CMA-ES and DE on three 10-dimensional problems below. Display mean best solution per generations (10 runs average) of each algorithm on shared line plot. For each problem parameterize the algorithms according to your own intuition. Choose evaluation-budget based termination criterion. In some cases logarythmic scale will be better for results comparison.

In [ ]:
rosenbrock_10 = get_problem("Rosenbrock", n_var=10)
ackley_10 = get_problem("Ackley", n_var=10)
schwefel_10 = get_problem("Schwefel", n_var=10)

termination = get_termination("n_eval", 1000)

In [ ]:
rosenbrock_visual = get_problem("Rosenbrock", n_var=2)
FitnessLandscape(rosenbrock_visual, angle=(45, 45), _type="surface").show()

In [ ]:
ackley_visual = get_problem("Ackley", n_var=2)
FitnessLandscape(ackley_visual, angle=(45, 45), _type="surface").show()
# Uncomment for alternative visualization:
# FitnessLandscape(ackley_visual, _type="contour", colorbar=True).show()

In [ ]:
schwefel_visual = get_problem("Schwefel", n_var=2)
FitnessLandscape(schwefel_visual, angle=(45, 45), _type="surface").show()
# Uncomment for alternative visualization:
# FitnessLandscape(schwefel_visual, _type="contour", colorbar=True).show()

### Excercise 5.1 - thorough benchmark
Choose a termination criterion based on target fuction value and compare the same algorithms on the same test instances. Select two function value targets pew function instance based on your knowledge from previous task.

#### What if an algorithm would never reach a target stopping value?
You can pass a tuple of termination criterions. The second criterion can be based on evaluation budget and serve as an upper bound. As for the performance comparison, use the Expected Running Time (ERT) metric which is calculated as follows:<br/>
ERT(function) = RT_s + (1 - p)/p * RT_us<br/>
where:
 - RT_s - average number of evaluations for succesful runs
 - RT_us - average number of evaluations for unsuccesful runs (in this case the upper bound)
  - p - fraction of succesful runs